In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [2]:
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

In [3]:
n = 400  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).partitionBy(n).persist()
edges = edges.map(lambda e: (e[1], e[0])).persist()

In [5]:
edges.count()

544304

In [4]:
pa = [[12,[12]]]
dest = 34
candidates = sc.parallelize(pa).partitionBy(n).persist()

In [5]:
def step(item):
    key=[item[1][-1],[item[0]]+ item[1][0]]
    
    return key

In [6]:
steps = 0
while edges.count() >0:
    print(steps)
    
    candidates = candidates.join(edges,n)
    if candidates.count() == 0:
        break
    
    edges = edges.leftOuterJoin(candidates,n).filter(lambda k: k[1][1] == None).map(lambda x: [x[0],x[1][0]]).persist()
    candidates = candidates.map(step).persist()
    m = candidates.filter(lambda x:x[0] == dest)
    if m.count()>0:
        print(m.collect())
        break
    steps+=1


0
1
2
3
4
5
6
7
[[34, [274, 23, 20, 107, 52, 53, 422, 12, 12]]]
